In [1]:
import torch as th
from torch import nn
import torch.nn.functional as F
#import torchvision
from torch.autograd import Variable
from torch import nn
from torch import optim
#from torchvision import datasets
#import torchvision.transforms as transforms
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import os
import pyprind

In [2]:
import os
root = 'data_ABIDE_useful (2)'
label_list = os.listdir(root)
import numpy as np
import pandas as pd
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import Data

class_dict = {
            "HC": 0,
            "ASD": 1,
            }
threshold = 0.2
data_processed = []
for label_files in label_list:
    label = torch.LongTensor([class_dict[label_files]])
    list = os.listdir(os.path.join(root, label_files))
    for files in list:
        subj_fc_dir = os.path.join(root, label_files, files)
        subj_mat_fc=np.loadtxt(subj_fc_dir)[:176,:90]
        #print("reading data " + subj_fc_dir)
        subj_mat_fc_adj = np.corrcoef(np.transpose(subj_mat_fc))#90*90--Correlation coefficient
        subj_mat_fc_adj = subj_mat_fc_adj - np.diag(np.diag(subj_mat_fc_adj))
        #print(subj_mat_fc_adj.shape)
        corr=subj_mat_fc_adj
        #take the upper triangle and compute the threshold
        subj_fc_adj_up=subj_mat_fc_adj[np.triu_indices(90,k=1)]
        subj_fc_adj_list = subj_fc_adj_up.reshape((-1))
        thindex = int(threshold * subj_fc_adj_list.shape[0])
        thremax = subj_fc_adj_list[subj_fc_adj_list.argsort()[-1 * thindex-1]]#
        #avoiding Nan
        subj_fc_adj_t = np.zeros((90, 90))
        subj_fc_adj_t[subj_mat_fc_adj > thremax] = 1
        subj_mat_fc_adj=subj_fc_adj_t
        fcedge_index, _ = dense_to_sparse(torch.from_numpy(subj_mat_fc_adj.astype(np.int16)))

        subj_mat_fc_list = subj_mat_fc.reshape((-1))
        subj_mat_fc_new = (subj_mat_fc - min(subj_mat_fc_list)) / (
                max(subj_mat_fc_list) - min(subj_mat_fc_list))
        # subj_mat_fc_new = (subj_mat_fc - np.mean(subj_mat_fc, axis=0, keepdims=True)) / np.std(subj_mat_fc, axis=0, keepdims=True)

        subj_mat_fc_new = np.transpose(subj_mat_fc_new)#90*176
                
        rowsum = np.array(subj_mat_fc_adj.sum(1))#row sum #90,
        N = np.diag(rowsum)#90*90
        degree_C_BOLD=np.concatenate((N,subj_mat_fc_new),1)#90*266
        #print(type(fcedge_index))
        #print(subj_mat_fc_adj)
        BOLD_C_degree=np.concatenate((subj_mat_fc_new,N),1)
        #print(type(BOLD_C_degree))
        ###one-hot###
        # subj_mat_fc_new=np.eye(args.num_nodes)
        data_processed.append(Data(x=torch.from_numpy(corr).float(), y=torch.tensor(label)))
        
import random
random.seed(1234)
random.shuffle(data_processed)
train_dataset = data_processed[:500]
test_dataset = data_processed[500:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

D:\Software\anaconda\envs\autism\lib\site-packages\ipykernel_launcher.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Number of training graphs: 500
Number of test graphs: 118


In [3]:
train_dataset

[Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90, 90], y=[1]),
 Data(x=[90,

In [4]:
import torch_geometric
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 32
Batch(batch=[2880], ptr=[33], x=[2880, 90], y=[32])

Step 2:
Number of graphs in the current batch: 32
Batch(batch=[2880], ptr=[33], x=[2880, 90], y=[32])

Step 3:
Number of graphs in the current batch: 32
Batch(batch=[2880], ptr=[33], x=[2880, 90], y=[32])

Step 4:
Number of graphs in the current batch: 32
Batch(batch=[2880], ptr=[33], x=[2880, 90], y=[32])

Step 5:
Number of graphs in the current batch: 32
Batch(batch=[2880], ptr=[33], x=[2880, 90], y=[32])

Step 6:
Number of graphs in the current batch: 32
Batch(batch=[2880], ptr=[33], x=[2880, 90], y=[32])

Step 7:
Number of graphs in the current batch: 32
Batch(batch=[2880], ptr=[33], x=[2880, 90], y=[32])

Step 8:
Number of graphs in the current batch: 32
Batch(batch=[2880], ptr=[33], x=[2880, 90], y=[32])

Step 9:
Number of graphs in the current batch: 32
Batch(batch=[2880], ptr=[33], x=[2880, 90], y=[32])

Step 10:
Number of graphs in the current batch: 32
Batch(batch=[2880], pt

In [132]:
class AutoEncoder(nn.Module):

    def __init__(self, inputDim, hiddenDim):
        super().__init__()
        self.inputDim = inputDim
        self.hiddenDim = hiddenDim
        self.encoder = nn.Linear(inputDim, hiddenDim, bias=True)
        self.decoder = nn.Linear(hiddenDim, inputDim, bias=True)
        self.act = F.sigmoid

    def forward(self, x, rep=False):

        hidden = self.encoder(x)
        hidden = self.act(hidden)
        if rep == False:# Perform layer-by-layer training
            out = self.decoder(hidden)
            #out = self.act(out)
            return out
        else:
            return hidden


class SAE(nn.Module):
    
    def __init__(self,encoderList):
        
        super().__init__()
        
        self.encoderList=encoderList
        self.en1 = encoderList[0]
        self.en2 = encoderList[1]
        self.en3 = encoderList[2]
        self.en4 = encoderList[3]
        '''
        self.en5 = encoderList[4]
        self.en6 = encoderList[5]
        self.en7 = encoderList[6]
        self.en8 = encoderList[7]
        self.en9 = encoderList[8]
        '''
        
        self.fc = nn.Linear(4000, 2, bias=True)
        
    def forward(self,x):
        
        out=x
        out = self.en1(out, rep=True)# overall training 
        out = self.en2(out, rep=True)
        out = self.en3(out, rep=True)
        out = self.en4(out, rep=True)
        '''
        out = self.en5(out, rep=True)
        out = self.en6(out, rep=True)
        out = self.en7(out, rep=True)
        out = self.en8(out, rep=True)
        out = self.en9(out, rep=True)
        '''
        out = self.fc(out)
        out = F.log_softmax(out)

        return out
    
class MLP(nn.Module):

    def __init__(self):

        super().__init__()
        
        self.fc1 = nn.Linear(8100, 5500, bias=True)
        self.fc2 = nn.Linear(5500, 4000, bias=True)
        #self.fc3 = nn.Linear(2100, 1000, bias=True)
        #self.fc4 = nn.Linear(1000, 500, bias=True)
        #self.fc5 = nn.Linear(500, 250, bias=True)
        #self.fc6 = nn.Linear(250, 50, bias=True)
        self.classify = nn.Linear(4000, 2, bias=True)
        self.act = F.sigmoid

    def forward(self, x):

        out = self.act(self.fc1(x))
        out = self.act(self.fc2(out))
        #out = self.act(self.fc3(out))
        #out = self.act(self.fc4(out))
        #out = self.act(self.fc5(out))
        #out = self.act(self.fc6(out))
        
        out = self.classify(out)
        out = F.log_softmax(out)

        return out

In [10]:
def trainAE(encoderList, trainLayer, batchSize, epoch, useCuda = False):

    if useCuda:
        for i in range(len(encoderList)):
            encoderList[i].cuda()

    optimizer = optim.SGD(encoderList[trainLayer].parameters(), lr=0.1)
    ceriation = nn.L1Loss()
    trainLoader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    testLoader = DataLoader(test_dataset, batch_size=32, shuffle=True)
    ##trainLoader, testLoader = loadMNIST(batchSize=batchSize)

    for i in range(epoch):

        sum_loss = 0

        if trainLayer != 0: # process layer 0 is separately cuz there is no predecessor encoder before the first encoder
            for i in range(trainLayer): # freeze all previous parameters to be trained
                for param in encoderList[i].parameters():
                    param.requires_grad = False

        for batch_idx, dt in enumerate(trainLoader):
            #(x, target)
            x=dt.x
            target=dt.y
            optimizer.zero_grad()
            if useCuda:
                x, target = x.cuda(), target.cuda()
            x, target = Variable(x), Variable(target)
            x = x.view(-1, 8100)
            # generate input data required for training layers
            out = x
            if trainLayer != 0:
                for i in range(trainLayer):
                    out = encoderList[i](out, rep=True)

            # train the specified autoencoder
            pred = encoderList[trainLayer](out, rep=False)

            loss = ceriation(pred, out)
            sum_loss += loss.item()
            loss.backward()
            optimizer.step()

            if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == len(trainLoader):
                print('==>>> train layer:{}, epoch: {}, batch index: {}, train loss: {:.6f}'
                      .format(trainLayer, i, batch_idx + 1, sum_loss/batch_idx))


def trainClassifier(model, batchSize, epoch, useCuda = False):

    if useCuda:
        model = model.cuda()

    # get para
    for param in model.parameters():
        param.requires_grad = True

    optimizer = optim.SGD(model.parameters(), lr=0.1)
    ceriation = nn.NLLLoss()
    trainLoader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    testLoader = DataLoader(test_dataset, batch_size=32, shuffle=True)
    ##trainLoader, testLoader = loadMNIST(batchSize=batchSize)
    
    for i in range(epoch):

        # trainning
        sum_loss = 0

        for batch_idx, dt in enumerate(trainLoader):
            #(x, target)
            x=dt.x
            target=dt.y
            optimizer.zero_grad()
            if useCuda:
                x, target = x.cuda(), target.cuda()
            x, target = Variable(x), Variable(target)
            x = x.view(-1, 8100)

            out = model(x)

            loss = ceriation(out, target.long())
            sum_loss += loss.item()
            loss.backward()
            optimizer.step()

            if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == len(trainLoader):
                print('==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format( i, batch_idx + 1, sum_loss/batch_idx))

        # testing
        correct_cnt, sum_loss = 0, 0
        total_cnt = 0
        for batch_idx, dt in enumerate(testLoader):
            #(x, target)
            x=dt.x
            target=dt.y

            x, target = Variable(x, volatile=True), Variable(target, volatile=True) # will not be used to update parameters
            
            if useCuda:
                x, target = x.cuda(), target.cuda()
            x = x.view(-1, 8100)

            out = model(x)
            print("out:",out)
            #print("target:",target)
            loss = ceriation(out, target)
            #print("loss:",loss)
            _, pred_label = th.max(out.data, 1)
            print("pred",pred_label)
            total_cnt += x.data.size()[0]
            correct_cnt += (pred_label == target.data).sum()

            # smooth average
            if (batch_idx + 1) % 100 == 0 or (batch_idx + 1) == len(testLoader):
                print('==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(
                    i, batch_idx + 1, sum_loss/batch_idx, correct_cnt * 1.0 / total_cnt))


In [133]:
if __name__ == '__main__':

    batchSize = 32
    AEepoch = 10
    epoch = 200
    
    print("trainlen:",len(train_loader))
    print("testlen:",len(test_loader))

    encoder1 = AutoEncoder(8100, 5500)
    encoder2 = AutoEncoder(5500, 5500)
    encoder3 = AutoEncoder(5500, 4000)
    en4 = AutoEncoder(4000, 4000)
    '''
    en5=AutoEncoder(500,250)
    en6=AutoEncoder(250,50)
    en7=AutoEncoder(900,600)
    en8=AutoEncoder(600,300)
    en9=AutoEncoder(300,40)
    '''
    encoderList = [encoder1, encoder2,encoder3,en4,]#en5,en6,en7,en8,en9,]

    trainAE(encoderList, 0, batchSize, AEepoch, useCuda=False)
    trainAE(encoderList, 1, batchSize, AEepoch, useCuda=False)
    trainAE(encoderList, 2, batchSize, AEepoch, useCuda=False)
    trainAE(encoderList, 3, batchSize, AEepoch, useCuda=False)
    '''
    trainAE(encoderList, 4, batchSize, AEepoch, useCuda=False)
    trainAE(encoderList, 5, batchSize, AEepoch, useCuda=False)
    trainAE(encoderList, 6, batchSize, AEepoch, useCuda=False)
    trainAE(encoderList, 7, batchSize, AEepoch, useCuda=False)
    trainAE(encoderList, 8, batchSize, AEepoch, useCuda=False)
    '''

trainlen: 16
testlen: 4
==>>> train layer:0, epoch: 0, batch index: 10, train loss: 0.341614
==>>> train layer:0, epoch: 0, batch index: 16, train loss: 0.324642
==>>> train layer:0, epoch: 1, batch index: 10, train loss: 0.326028
==>>> train layer:0, epoch: 1, batch index: 16, train loss: 0.309393
==>>> train layer:0, epoch: 2, batch index: 10, train loss: 0.310440
==>>> train layer:0, epoch: 2, batch index: 16, train loss: 0.295195
==>>> train layer:0, epoch: 3, batch index: 10, train loss: 0.296093
==>>> train layer:0, epoch: 3, batch index: 16, train loss: 0.282089
==>>> train layer:0, epoch: 4, batch index: 10, train loss: 0.283440
==>>> train layer:0, epoch: 4, batch index: 16, train loss: 0.269810
==>>> train layer:0, epoch: 5, batch index: 10, train loss: 0.271745
==>>> train layer:0, epoch: 5, batch index: 16, train loss: 0.258522
==>>> train layer:0, epoch: 6, batch index: 10, train loss: 0.260375
==>>> train layer:0, epoch: 6, batch index: 16, train loss: 0.248142
==>>> trai

In [134]:
epoch = 200
model = SAE(encoderList)
#model = MLP()
print(model)

trainClassifier(model, batchSize, epoch, useCuda=False)

SAE(
  (en1): AutoEncoder(
    (encoder): Linear(in_features=8100, out_features=5500, bias=True)
    (decoder): Linear(in_features=5500, out_features=8100, bias=True)
  )
  (en2): AutoEncoder(
    (encoder): Linear(in_features=5500, out_features=5500, bias=True)
    (decoder): Linear(in_features=5500, out_features=5500, bias=True)
  )
  (en3): AutoEncoder(
    (encoder): Linear(in_features=5500, out_features=4000, bias=True)
    (decoder): Linear(in_features=4000, out_features=5500, bias=True)
  )
  (en4): AutoEncoder(
    (encoder): Linear(in_features=4000, out_features=4000, bias=True)
    (decoder): Linear(in_features=4000, out_features=4000, bias=True)
  )
  (fc): Linear(in_features=4000, out_features=2, bias=True)
)
==>>> epoch: 0, batch index: 10, train loss: 2.217575
==>>> epoch: 0, batch index: 16, train loss: 2.397546
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 12, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 13, batch index: 10, train loss: 1.704246
==>>> epoch: 13, batch index: 16, train loss: 1.361430
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 13, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 14, batch index: 10, train loss: 1.376685
==>>> epoch: 14, batch index: 16, train loss: 1.285056
pred tensor([1, 1,

==>>> epoch: 26, batch index: 16, train loss: 0.979292
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 26, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 27, batch index: 10, train loss: 0.837534
==>>> epoch: 27, batch index: 16, train loss: 0.780751
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0

pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 39, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 40, batch index: 10, train loss: 0.947275
==>>> epoch: 40, batch index: 16, train loss: 0.863677
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 40, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 41, batch index: 10, train loss: 0.886805
==>>> epoch: 41, batch index: 16, train loss: 0.822837
pred tensor([0, 0,

==>>> epoch: 53, batch index: 16, train loss: 0.862481
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 53, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 54, batch index: 10, train loss: 0.843724
==>>> epoch: 54, batch index: 16, train loss: 0.786683
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0

...

==>>> epoch: 134, batch index: 10, train loss: 0.782779
==>>> epoch: 134, batch index: 16, train loss: 0.766029
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 134, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 135, batch index: 10, train loss: 0.783640
==>>> epoch: 135, batch index: 16, train loss: 0.761745
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 

pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 147, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 148, batch index: 10, train loss: 0.799391
==>>> epoch: 148, batch index: 16, train loss: 0.765258
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 148, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 149, batch index: 10, train loss: 0.840687
==>>> epoch: 149, batch index: 16, train loss: 0.797934
pred tensor(

==>>> epoch: 161, batch index: 10, train loss: 0.796767
==>>> epoch: 161, batch index: 16, train loss: 0.756693
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 161, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 162, batch index: 10, train loss: 0.825198
==>>> epoch: 162, batch index: 16, train loss: 0.774838
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 

pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 174, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 175, batch index: 10, train loss: 0.792697
==>>> epoch: 175, batch index: 16, train loss: 0.761582
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 175, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 176, batch index: 10, train loss: 0.772943
==>>> epoch: 176, batch index: 16, train loss: 0.742804
pred tensor(

==>>> epoch: 188, batch index: 10, train loss: 0.797213
==>>> epoch: 188, batch index: 16, train loss: 0.763536
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 188, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 189, batch index: 10, train loss: 0.787094
==>>> epoch: 189, batch index: 16, train loss: 0.749391
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [15]:
#===========%==========
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8100, 6700, bias=True)
        self.fc2 = nn.Linear(6700, 5500, bias=True)
        self.fc3 = nn.Linear(5500, 5000, bias=True)
        #self.fc4 = nn.Linear(1000, 500, bias=True)
        #self.fc5 = nn.Linear(500, 250, bias=True)
        #self.fc6 = nn.Linear(250, 50, bias=True)
        self.classify = nn.Linear(5000, 2, bias=True)
        self.act = F.sigmoid
    def forward(self, x):
        out = self.act(self.fc1(x))
        out = self.act(self.fc2(out))
        out = self.act(self.fc3(out))
        #out = self.act(self.fc4(out))
        #out = self.act(self.fc5(out))
        #out = self.act(self.fc6(out))
        out = self.classify(out)
        out = F.log_softmax(out)
        return out
    
batchSize = 32
epoch = 2
model = MLP()
print(model)
trainClassifier(model, batchSize, epoch, useCuda=False)

MLP(
  (fc1): Linear(in_features=8100, out_features=6700, bias=True)
  (fc2): Linear(in_features=6700, out_features=5500, bias=True)
  (fc3): Linear(in_features=5500, out_features=5000, bias=True)
  (classify): Linear(in_features=5000, out_features=2, bias=True)
)


D:\Software\anaconda\envs\autism\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


==>>> epoch: 0, batch index: 10, train loss: 10.404205
==>>> epoch: 0, batch index: 16, train loss: 6.934919
out: tensor([[-1.1675, -0.3727],
        [-1.1680, -0.3725],
        [-1.1684, -0.3723],
        [-1.1678, -0.3726],
        [-1.1676, -0.3727],
        [-1.1682, -0.3724],
        [-1.1676, -0.3727],
        [-1.1666, -0.3731],
        [-1.1669, -0.3730],
        [-1.1675, -0.3727],
        [-1.1674, -0.3728],
        [-1.1678, -0.3726],
        [-1.1683, -0.3724],
        [-1.1679, -0.3725],
        [-1.1678, -0.3726],
        [-1.1683, -0.3724],
        [-1.1675, -0.3727],
        [-1.1675, -0.3727],
        [-1.1678, -0.3726],
        [-1.1678, -0.3726],
        [-1.1674, -0.3728],
        [-1.1674, -0.3728],
        [-1.1676, -0.3727],
        [-1.1680, -0.3725],
        [-1.1680, -0.3725],
        [-1.1678, -0.3726],
        [-1.1678, -0.3726],
        [-1.1686, -0.3722],
        [-1.1676, -0.3727],
        [-1.1679, -0.3726],
        [-1.1679, -0.3725],
        [-1.1678, 

D:\Software\anaconda\envs\autism\lib\site-packages\ipykernel_launcher.py:98: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


out: tensor([[-1.1680, -0.3725],
        [-1.1680, -0.3725],
        [-1.1676, -0.3727],
        [-1.1677, -0.3726],
        [-1.1671, -0.3729],
        [-1.1677, -0.3726],
        [-1.1675, -0.3727],
        [-1.1676, -0.3727],
        [-1.1676, -0.3727],
        [-1.1668, -0.3730],
        [-1.1677, -0.3726],
        [-1.1677, -0.3726],
        [-1.1682, -0.3724],
        [-1.1677, -0.3726],
        [-1.1678, -0.3726],
        [-1.1673, -0.3728],
        [-1.1672, -0.3728],
        [-1.1675, -0.3727],
        [-1.1674, -0.3728],
        [-1.1676, -0.3727],
        [-1.1674, -0.3728],
        [-1.1677, -0.3726],
        [-1.1682, -0.3724],
        [-1.1669, -0.3730],
        [-1.1681, -0.3725],
        [-1.1671, -0.3729],
        [-1.1679, -0.3725],
        [-1.1683, -0.3724],
        [-1.1676, -0.3727],
        [-1.1676, -0.3727],
        [-1.1676, -0.3727],
        [-1.1679, -0.3726]], grad_fn=<LogSoftmaxBackward>)
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [127]:
'''
## self.fc1 = nn.Linear(8100, 5500, bias=True)
#        self.fc2 = nn.Linear(5500, 4000, bias=True)
#        self.classify = nn.Linear(4000, 2, bias=True)
#========66.9%============
'''
epoch = 200
#model = SAE(encoderList)
model = MLP()
print(model)

trainClassifier(model, batchSize, epoch, useCuda=False)

MLP(
  (fc1): Linear(in_features=8100, out_features=5500, bias=True)
  (fc2): Linear(in_features=5500, out_features=4000, bias=True)
  (classify): Linear(in_features=4000, out_features=2, bias=True)
)
==>>> epoch: 0, batch index: 10, train loss: 1.845297
==>>> epoch: 0, batch index: 16, train loss: 2.208474
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 0, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 1, batch index: 10, train loss: 2.158240
==>>> epoch: 1, batch index: 16, train loss: 2.132996
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,


==>>> epoch: 14, batch index: 10, train loss: 1.126302
==>>> epoch: 14, batch index: 16, train loss: 1.281592
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 14, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 15, batch index: 10, train loss: 0.977457
==>>> epoch: 15, batch index: 16, train loss: 1.121048
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

==>>> epoch: 28, batch index: 10, train loss: 0.838243
==>>> epoch: 28, batch index: 16, train loss: 0.778525
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 28, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 29, batch index: 10, train loss: 0.868782
==>>> epoch: 29, batch index: 16, train loss: 0.854341
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

==>>> epoch: 42, batch index: 10, train loss: 0.796360
==>>> epoch: 42, batch index: 16, train loss: 0.763190
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 42, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 43, batch index: 10, train loss: 0.790762
==>>> epoch: 43, batch index: 16, train loss: 0.749387
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

==>>> epoch: 56, batch index: 10, train loss: 0.877903
==>>> epoch: 56, batch index: 16, train loss: 0.813130
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 56, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 57, batch index: 10, train loss: 0.886280
==>>> epoch: 57, batch index: 16, train loss: 0.818021
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

==>>> epoch: 140, batch index: 10, train loss: 0.795127
==>>> epoch: 140, batch index: 16, train loss: 0.761263
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 140, batch index: 4, test loss: 0.000000, acc: 0.500
==>>> epoch: 141, batch index: 10, train loss: 0.817753
==>>> epoch: 141, batch index: 16, train loss: 0.763001
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 

==>>> epoch: 154, batch index: 10, train loss: 0.787145
==>>> epoch: 154, batch index: 16, train loss: 0.764570
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 154, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 155, batch index: 10, train loss: 0.783509
==>>> epoch: 155, batch index: 16, train loss: 0.740247
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 

==>>> epoch: 168, batch index: 10, train loss: 0.787280
==>>> epoch: 168, batch index: 16, train loss: 0.741500
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 168, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 169, batch index: 10, train loss: 0.760881
==>>> epoch: 169, batch index: 16, train loss: 0.729930
pred tensor([1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1])
pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 

==>>> epoch: 182, batch index: 10, train loss: 0.761815
==>>> epoch: 182, batch index: 16, train loss: 0.748128
pred tensor([1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
        1, 0, 1, 0, 1, 0, 0, 0])
pred tensor([0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
        1, 0, 1, 1, 1, 0, 1, 0])
pred tensor([1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 0, 1])
pred tensor([1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1])
==>>> epoch: 182, batch index: 4, test loss: 0.000000, acc: 0.644
==>>> epoch: 183, batch index: 10, train loss: 0.768020
==>>> epoch: 183, batch index: 16, train loss: 0.729864
pred tensor([1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 0, 1, 0])
pred tensor([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 1])
pred tensor([0, 1, 1, 1, 1, 1, 1, 1, 0, 

pred tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
==>>> epoch: 195, batch index: 4, test loss: 0.000000, acc: 0.525
==>>> epoch: 196, batch index: 10, train loss: 0.768153
==>>> epoch: 196, batch index: 16, train loss: 0.738101
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
==>>> epoch: 196, batch index: 4, test loss: 0.000000, acc: 0.475
==>>> epoch: 197, batch index: 10, train loss: 0.755411
==>>> epoch: 197, batch index: 16, train loss: 0.731032
pred tensor([0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1])
pred tensor(